In [1]:
#Just a random change

#calculating distance: gridrefGDF['DistToRiver'] = gridrefGDF["geometry"].apply(lambda x: df.distance(x).min())
#season

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import os
import glob
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import seaborn as sns
import pyproj
import geopandas as gpd
import fiona
import shapely
from shapely import LineString, MultiPolygon
from shapely.geometry import Point
from sklearn.metrics.pairwise import euclidean_distances
import math
from math import cos, asin, sqrt
import csv
from shapely.ops import unary_union

In [3]:
#def import class
def import_csv(input_no, folder_path, csv_files):
    if input_no < 1 or input_no > len(csv_files):
        raise ValueError("input_no should be between 1 and the number of CSV files")
    # Create the full path for each CSV file
    full_paths = [os.path.join(folder_path, csv) for csv in csv_files[:input_no]]

    # Read CSV files into DataFrames
    dfs = [pd.read_csv(full_path, dtype={"INTERPRETATION": str, "TEXT_RESULT": str, "DETCODE": str, "PTCODE":str}) for full_path in full_paths]

    # Concatenate DataFrames
    full_wims = pd.concat(dfs)

    return full_wims

## Getting rid of any duplicate rows

In [4]:
def remove_duplicate_rows(full_wims_all):
    duplicate_rows = full_wims_all[full_wims_all.duplicated()]
    if duplicate_rows.empty:
        print("No duplicate rows found.")
        full_wims_no_dup = full_wims_all
    else:
        num_duplicates = len(duplicate_rows)
        num_wims = len(full_wims_all)
        print(f"Number of duplicate rows found: {num_duplicates}")
        print(f"Total number of rows in dataset: {num_wims}")
        full_wims_no_dup = full_wims_all.drop_duplicates()
        num_no_dup = len(full_wims_no_dup)
        print(f"Total number of rows in dataset after duplicates removed: {num_no_dup}")

    return full_wims_no_dup

In [5]:
def read_shapefiles(folder_path):
    # Find all shapefiles in the specified folder
    shapefile_paths = glob.glob(os.path.join(folder_path, '*.shp'))

    if not shapefile_paths:
        raise FileNotFoundError("No shapefiles found in the specified folder.")

    # If there is only one shapefile, read it directly
    if len(shapefile_paths) == 1:
        return gpd.read_file(shapefile_paths[0])

    # If there are multiple shapefiles, read and union them
    gdf_list = [gpd.read_file(shapefile) for shapefile in shapefile_paths]
    combined_shapefile = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True), crs=gdf_list[0].crs)

    return combined_shapefile

In [6]:
def extract_shapefile(data_dict, estuary_to_extract, input_shapefile, target_crs='EPSG:xxxx'):
    if estuary_to_extract in data_dict:
        # Get the corresponding values from the dictionary
        corresponding_values = data_dict[estuary_to_extract]

        # Initialize an empty GeoDataFrame
        combined_gdf = gpd.GeoDataFrame()
    
        # Iterate through each corresponding value
        for value in corresponding_values:
            # Filter rows based on the current value
            subset_gdf = input_shapefile[input_shapefile['wb_name'] == value]

            # Check if the subset is not empty before combining
            if not subset_gdf.empty:
                # Concatenate the subset to the combined GeoDataFrame
                combined_gdf = pd.concat([combined_gdf, subset_gdf], ignore_index=True)

        # Check if there is more than one row for the key
        if len(combined_gdf) > 1:
            # Use unary_union to combine geometries
            combined_geometry = unary_union(combined_gdf['geometry'])

            # Create a new GeoDataFrame with the combined geometry
            new_gdf = gpd.GeoDataFrame(geometry=[combined_geometry], crs=target_crs)

            # Print or do further processing with the new GeoDataFrame
            print(new_gdf)
        elif not combined_gdf.empty:
            # Only one row, no need to combine, use the original GeoDataFrame
            new_gdf = combined_gdf.copy()
            new_gdf.crs = target_crs  # Set CRS for the single-row GeoDataFrame
            print(new_gdf)
        else:
            print(f"No features found for {estuary_to_extract}.")
    else:
        print(f"{estuary_to_extract} not found in the dictionary.")

    return new_gdf


In [7]:
def find_longest_line(shape): #set indices
    
    longest_distance = 0
    longest_line = None

    #extracting the longest line from the shapefile
    for index, row in shape.iterrows():
        polygon = row['geometry']
        exterior_ring = polygon.exterior
        #This doesn't exist in Plymouth but no idea about other estuaries
        for interior_ring in polygon.interiors:
            ring = interior_ring
            for i in range(len(ring.coords) - 1):
                point1 = ring.coords[i]
                point2 = ring.coords[i + 1]
                line = LineString([point1, point2])
                distance = line.length
                #Replace longest distance
                if distance > longest_distance:
                    longest_distance = distance
                    longest_line = line
    
        for i in range(len(exterior_ring.coords) - 1):
            point1 = exterior_ring.coords[i]
            point2 = exterior_ring.coords[i + 1]
            line = LineString([point1, point2])
            distance = line.length
            if distance > longest_distance:
                longest_distance = distance
                longest_line = line
                
    reprojected_longest_line_gdf = gpd.GeoDataFrame({'geometry': [longest_line]}, geometry='geometry', crs="EPSG:4326")
    return reprojected_longest_line_gdf


In [8]:
def redefine_site_number(estuary, input_data, longest_line, est_shape, output_folder_path):
    #define the transformer
    crs_british = pyproj.CRS('EPSG:27700')
    target_crs = pyproj.CRS('EPSG:4326')

    #the always_xy argument tells the transformer how to expect the easting/northing variables
    transformer = pyproj.Transformer.from_crs(crs_british, target_crs, always_xy=True)

    #create copy
    full_wims_reproj = input_data.copy()

    longitude, latitude = transformer.transform(full_wims_reproj['easting'], full_wims_reproj['northing'])
    full_wims_reproj['lat'] = latitude
    full_wims_reproj['lon'] = longitude

    # Create a GeoDataFrame with list of unique points from lon and lat columns
    sites = full_wims_reproj[['lon', 'lat']].drop_duplicates()
    
    # Create Point geometries using 'lon' and 'lat'
    geometry = [Point(xy) for xy in zip(sites['lon'], sites['lat'])]

    # Create the GeoDataFrame
    sites_gdf = gpd.GeoDataFrame(sites, geometry=geometry)

    # Set the CRS of the GeoDataFrame
    sites_gdf.crs = "EPSG:4326"

    # Reproject the 'sites_gdf' GeoDataFrame to the same CRS as 'reprojected_longest_line_gdf'
    sites_reprojected = sites_gdf.to_crs(longest_line.crs)

    # Calculate distances to the reprojected longest line
    sites_reprojected['DistToEstuary'] = sites_reprojected['geometry'].apply(
        lambda x: longest_line["geometry"].distance(x).min()
    )

    # Sort the GeoDataFrame
    sites_reprojected.sort_values('DistToEstuary', inplace=True)
    sites_reprojected.reset_index(drop=True, inplace=True)

    # Add the 'site_number' column
    sites_reprojected['site_number'] = sites_reprojected.index + 1
    sites_reprojected['site_number'] = sites_reprojected['site_number'].astype(int)

    #Merge on left to link full_wims to sites_reprojected
    full_wims_reproj = full_wims_reproj.merge(sites_reprojected, on =['lon','lat'], how = 'left')


    fig, ax0 = plt.subplots(nrows=1, ncols=1,figsize=(12, 10))
    
    est_shape.plot(color='lightgray',ax=ax0)
    
    scatter = ax0.scatter(full_wims_reproj['lon'], full_wims_reproj['lat'], c=full_wims_reproj['site_number'],
                          edgecolor="none", s=70)
    longest_line.plot(color='red',ax=ax0)
    
    # Set the title and labels
    ax0.set_title(f'Site map for {estuary}')
    ax0.set_xlabel('Easting (m)')
    ax0.set_ylabel('Northing (m)')

    colorbar = plt.colorbar(scatter)
    colorbar.set_label('Site Number')
    
    output_plot_path = os.path.join(output_folder_path, f'site_map_{estuary}.png')
    plt.savefig(output_plot_path, bbox_inches='tight')
    
    plt.close()

    return full_wims_reproj

In [9]:
def convert_date_times(full_wims_reproj):
    full_wims_dt = full_wims_reproj.copy()
    full_wims_dt['date']= pd.to_datetime(full_wims_dt['date'], dayfirst=True,format='mixed')
    #Get year
    full_wims_dt['year'] = full_wims_dt['date'].dt.strftime('%Y')
    #Get month
    full_wims_dt['month'] = full_wims_dt['date'].dt.strftime('%m')
    #Get day
    full_wims_dt['day'] = full_wims_dt['date'].dt.strftime('%d')
    #Get month/day
    full_wims_dt['month_day'] = full_wims_dt['date'].dt.strftime('%m-%d')
    #Convert year to number
    full_wims_dt['year'] = full_wims_dt['year'].astype('int')
    #Convert day to number
    full_wims_dt['day'] = full_wims_dt['day'].astype('int')
    print(len(full_wims_dt))
    print('Dates converted successfully!')
    return full_wims_dt

In [10]:
def screen_sample_depths(estuary,full_wims_dt,output_folder_path):
    #Create unique ID to extract sample depth
    full_wims_dt['unique_key'] = full_wims_dt['id'].astype(str) + full_wims_dt['sampno'].astype(str) + full_wims_dt['time'].astype(str)
    #Select only rows with sample depth info
    sample_depth = full_wims_dt[full_wims_dt['detname']=='Sample Depth below surface']
    #Remov
    sample_numbers_with_non_zero_depth = sample_depth[sample_depth['result'] > 1]['unique_key'].tolist()
    
    full_wims_surface_samples = full_wims_dt.copy()
    full_wims_surface_samples = full_wims_surface_samples[~full_wims_surface_samples['unique_key'].isin(sample_numbers_with_non_zero_depth)]
    
    sample_depth_after_filter = full_wims_surface_samples[full_wims_surface_samples['detname']=='Sample Depth below surface']

    fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2,figsize=(18, 10))
    
    ax0.hist(sample_depth['result'], bins='auto', edgecolor='black', log=True)
    ax0.set_title(f'{estuary}: Histogram of sample depths before filtering')
    ax0.set_xlabel('Sample depth (m)')
    ax0.set_ylabel('Number of samples')

    ax1.hist(sample_depth_after_filter['result'], bins='auto', edgecolor='black', log=True)
    ax1.set_title(f'{estuary}: Histogram of sample depths after filtering')
    ax1.set_xlabel('Sample depth (m)')
    ax1.set_ylabel('Number of samples')
    
    output_plot_path = os.path.join(output_folder_path, f'sample_depth_histogram_{estuary}.png')
    plt.savefig(output_plot_path, bbox_inches='tight')
    
    plt.close()

    return full_wims_surface_samples

In [11]:
def get_seasons(full_wims_surface_samples):
    full_wims_seasons = full_wims_surface_samples.copy()

    #found out roughly when the first and last day of each season is - this changes year to year though
    full_wims_seasons['astronomic season'] = ''
    full_wims_seasons['astronomic season'] = np.where((full_wims_seasons['month_day'] >= '03-30') & (full_wims_seasons['month_day'] <= '06-22'), 'Spring', full_wims_seasons['astronomic season'])
    full_wims_seasons['astronomic season'] = np.where((full_wims_seasons['month_day'] > '06-22') & (full_wims_seasons['month_day'] <= '09-23'), 'Summer', full_wims_seasons['astronomic season'])
    full_wims_seasons['astronomic season'] = np.where((full_wims_seasons['month_day'] > '09-23') & (full_wims_seasons['month_day'] <'12-22'), 'Autumn', full_wims_seasons['astronomic season'])
    full_wims_seasons['astronomic season'].replace('','Winter',inplace=True)
    
    return full_wims_seasons

In [12]:
#Salinity calcs
def pythagorean(lat1, lon1, lat2, lon2):
    return np.sqrt((lat2-lat1)**2 + (lon2-lon1)**2)

def closest(data, v):
    if not data:
        # Handle the case when data is empty (e.g., return a default value or raise an exception)
        return None  # Or any other appropriate action

    return min(data, key=lambda p: pythagorean(v['lat'], v['lon'], p['lat'], p['lon']))
    
def assign_average_salinity(estuary,full_wims_seasons,output_folder_path):
    wims_sal = full_wims_seasons[full_wims_seasons['detcode'].str.contains('6687|7608|0063')]
    #Removal of any brine samples
    wims_sal = wims_sal[wims_sal['result'] <= 50] # Removed before salinity grouping to stop outliers changing groupings.
    #copy dataframe
    wims_sal_group = wims_sal.copy()
    #group salinity data by mean for site number
    wims_sal_group = wims_sal_group.groupby('site_number')['result'].mean()
    #Merge the salinity average value on the grouped salinity
    wims_sal_avg = wims_sal.merge(wims_sal_group, on = 'site_number', how='left')
    #The results column will be duplicated, hence rename the second one site_avg_salinity
    wims_sal_avg.rename(columns={'result_y':'site_avg_salinity'},inplace=True)

    #Define the salinity boundary conditions and assign them to a value
    conditions = [
        wims_sal_avg['site_avg_salinity'] < 0.5,
        (wims_sal_avg['site_avg_salinity'] >= 0.5) & (wims_sal_avg['site_avg_salinity'] < 30)
    ]

    choices = ['freshwater', 'brackish']
    wims_sal_avg['salinity_class'] = np.select(conditions, choices, default='saline')
    #Remove excess salinity class counts
    wims_sal_avg['salinity_class'].value_counts(dropna=False)
    #Take the site numbers
    unique_sal_values = wims_sal_avg[['site_number','site_avg_salinity','salinity_class']].drop_duplicates()
    #Takes a copy of the dataset
    full_wims_sal = full_wims_seasons.copy()
    #Merge onto the main dataframe to assign any values where there is a dataset
    full_wims_sal = pd.merge(full_wims_sal, unique_sal_values, on=['site_number'], how='left')
    
    #Take a copy of the dataset
    full_wims_sal_num_class = full_wims_sal.copy()
    full_wims_sal_num_class['salinity_class'].replace({'freshwater':0.0,'brackish':1.0,'saline':2.0},inplace=True)
    full_wims_sal_num_class['salinity_class'].value_counts(dropna=False)

    #Grab the latitude and longitude for any sample points where there IS NOT a salinity class
    missing_salinity_points = full_wims_sal_num_class[full_wims_sal_num_class['salinity_class'].isna()][['lat','lon']].drop_duplicates()
    #Grab the latitude and longitude for any sample points where there IS a salinity class
    available_salinity_points = full_wims_sal_num_class[full_wims_sal_num_class['salinity_class'].notna()][['lat','lon']].drop_duplicates()

    # Create a new DataFrame with closest points
    result_data2 = []

    #Check to see if there are available salinity points
    if not available_salinity_points.empty:
        #Check to see if there are any missing salinity points
        if not missing_salinity_points.empty:
            for index, row in missing_salinity_points.iterrows():
                closest_point = closest(available_salinity_points.to_dict(orient='records'), row.to_dict())
                if closest_point is not None:
                    result_data2.append({
                        'missing_sal_lat': row['lat'],
                        'missing_sal_lon': row['lon'],
                        'closest_lat': closest_point['lat'],
                        'closest_lon': closest_point['lon'],
                        })
                else:
                    # Handle the case when there are no closest salinity points (e.g., skip, print a message, or raise an exception)
                    print(f"No closest salinity point found for coordinates {row['lat']}, {row['lon']}")

            # Take the resulting dictionary and convert to DataFrame
            result_df2 = pd.DataFrame(result_data2)
    
            #look in the dataset for where there is salinity measurements
            available_salinity_samples = full_wims_sal_num_class[full_wims_sal_num_class['salinity_class'].notna()][['lat','lon','site_avg_salinity','salinity_class','site_number']].drop_duplicates()
            available_salinity_samples.rename(columns={'salinity_class':'replaced_salinity'},inplace=True)
            available_salinity_samples.rename(columns={'site_avg_salinity':'replaced_site_avg_salinity'},inplace=True)
    
            #Add the data from available_salinity_samples to the nearest lat_long to each missing sample
            filled_nas = pd.merge(result_df2, available_salinity_samples, left_on=['closest_lat','closest_lon'], right_on=['lat','lon'])
        
            #missing_lat and missing_lon are the coordinates from rows missing salinities in full wims
            #we can drop the other columns now because they were just used for the join to grab
            #nearest salinity class and site number
            filled_nas = filled_nas[['missing_sal_lat','missing_sal_lon','replaced_site_avg_salinity','replaced_salinity','site_number']]
    
            #Merge the salinity value from filled_nas with the full_wims_sal created above. This will not occur for any samples where salinity already exists
            salinity_filled = pd.merge(full_wims_sal_num_class, filled_nas,  left_on=['lat','lon'], right_on=['missing_sal_lat','missing_sal_lon'], how='left')
            #Fill the salinity classification
            salinity_filled['salinity_class'] = salinity_filled['salinity_class'].fillna(salinity_filled['replaced_salinity'])
            salinity_filled['site_avg_salinity'] = salinity_filled['site_avg_salinity'].fillna(salinity_filled['replaced_site_avg_salinity'])
            #checking to see that every site has a salinity class
            salinity_filled['salinity_class'].value_counts(dropna=False)
            #Drop unnecessary columns
            salinity_filled.drop(columns=[ 'missing_sal_lon', 'replaced_salinity', 'site_number_y','replaced_site_avg_salinity'],inplace=True)
            #Rename the site number
            salinity_filled.rename(columns={'site_number_x':'site_number'},inplace=True)
            #Copy the dataset
            wims_sal_filled = salinity_filled.copy()
            #turning classifications back into label/string format because nan values are gone now
            wims_sal_filled['salinity_class'].replace({0:'freshwater', 1:'brackish', 2:'saline'},inplace=True)

            #Applying a column to highlight if salinity classification was calculated or taken using nearest neighbour
            # Replace numeric values with 'Nearest Neighbour'. Checking for not null and a numeric value
            wims_sal_filled.loc[wims_sal_filled['missing_sal_lat'].notnull() & wims_sal_filled['missing_sal_lat'].apply(lambda x: isinstance(x, (int, float))), 'missing_sal_lat'] = 'Nearest Neighbour'
            # Fill remaining gaps with 'Calculated'
            wims_sal_filled['missing_sal_lat'].fillna('Calculated', inplace=True)
            # Rename the column
            wims_sal_filled_renamed = wims_sal_filled.rename(columns={'missing_sal_lat': 'sal_source'})
        
        else:
            wims_sal_filled_renamed = full_wims_sal_num_class.copy()
            print('No missing salinity points to fill') 
    else:
        wims_sal_filled_renamed = full_wims_sal_num_class.copy()
        print('No available salinity points to fill with') 


    fig, (ax0, ax1) = plt.subplots(nrows=2, ncols=1,figsize=(20, 10))
    
    shapefile.plot(color='lightgray', ax=ax0)
    sns.scatterplot(x='easting', y='northing', palette='husl', hue='salinity_class', data=wims_sal_avg,
                                 edgecolor="none", s=70, ax= ax0)
    ax0.set_title('Geographical Points Salinity Classifications Before Nearest Neighbour Fill')

    shapefile.plot(color='lightgray', ax=ax1)
    sns.scatterplot(x='easting', y='northing', palette='husl', hue='salinity_class', data=wims_sal_filled_renamed,
                                 edgecolor="none", s=70, ax=ax1)
    ax1.set_title('Geographical Points Salinity Classifications After Nearest Neighbour Fill')

    output_plot_path = os.path.join(output_folder_path, f'salinity_map_{estuary}.png')
    plt.savefig(output_plot_path, bbox_inches='tight')
    plt.close()
    return wims_sal_filled_renamed
    

In [13]:
def generate_seperate_dataframes(wims_sal_filled_renamed):
    wims_nd_removed = wims_sal_filled_renamed[wims_sal_filled_renamed['qual'].isna()] #this means there weren't any greater than or less than symbols in these rows
    wims_nd_converted = wims_sal_filled_renamed.copy()
    wims_nd_converted['result'] = wims_nd_converted.apply(lambda row: row['result'] * 0.5 if row['qual'] == '<' else row['result'], axis=1)
    print(len(wims_nd_converted))
    print(len(wims_nd_removed))
    #Making a column that combines the det codes and labels them either as one of our 
    #chosen metrics or "Other" for filtering purposes
    wims_nd_converted['param'] = 'Other'
    wims_nd_removed['param'] = 'Other'

    for index, value_df in wims_nd_converted['detcode'].items():
        for key, value_list in params.items():
            if value_df in value_list:
                wims_nd_converted.loc[index, 'param'] = key
                break

    for index, value_df in wims_nd_removed['detcode'].items():
        for key, value_list in params.items():
            if value_df in value_list:
                wims_nd_removed.loc[index, 'param'] = key
                break

    #need to fix this and make it clean above?
    wims_nd_converted.rename(columns={ 'site_number_x':'site_number'},inplace=True)
    wims_nd_removed.rename(columns={ 'site_number_x':'site_number'},inplace=True)

    return wims_nd_converted, wims_nd_removed

In [14]:
#dictionary of detcodes
params = {'Temperature': ['0076'], 'pH': ['0061'], 'Dissolved Oxygen':['9924'],'Dissolved Oxygen Saturation':['9901'],
          'Suspended Solids at 105C':['0135'], 'Salinity':['1198', '3028', '7608', '7609', '6687'],
          'Ammoniacal Nitrogen': ['0111', '9993'], 'Nitrate': ['0117', '9853'], 'Nitrite':['0118', '6485'],
          'Biological Oxygen Demand': ['0085', '0088'], 'Copper': ['6450', '6452'], 'Zinc':['3408', '6455'],
          'Orthophosphate':['0180', '9856'], 'Alkalinity': ['0162'], 'Turbidity':['6396', '3976']}

#dictionary of hexcodes
colours = {'Temperature': '#f77189', 'pH': '#dc8932', 'Dissolved Oxygen': '#ae9d31','Dissolved Oxygen Saturation': '#ae9d31',
          'Suspended Solids at 105C': '#77ab31', 'Salinity':'#36ada4',
          'Ammoniacal Nitrogen': '#4878d0', 'Nitrate': '#cc7af4', 'Nitrite':'#f565cc',
          'Biological Oxygen Demand': '#21ada8', 'Copper': '#ff7f00', 'Zinc':'#b2df8a',
          'Orthophosphate':'#6a3d9a', 'Alkalinity': '#003399', 'Turbidity':'#b30086'}

In [15]:
def plot_all_variables(estuary, nd_converted, nd_removed, colours, output_folder_path, filter_title):
    for i,j in zip(colours.keys(),colours.values()):
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))

        #scatter plot
        sns.scatterplot(x='site_number', y='result', data=nd_converted[nd_converted['param'] == i], color='black', ax=axes[0])
        sns.scatterplot(x='site_number', y='result', data=nd_removed[nd_removed['param'] == i], color=j, ax=axes[0])
        axes[0].set_title('Scatter Plot')
        axes[0].set_xlabel('Site ID')
        axes[0].set_ylabel(f'{i}')

        #kde plot
        sns.kdeplot(x='result', data=nd_converted[nd_converted['param'] == i], fill=True, color='black', ax=axes[1])
        sns.kdeplot(x='result', data=nd_removed[nd_removed['param'] == i], fill=True, color=j, ax=axes[1])
        axes[1].set_title('KDE Plot')
        axes[1].set_xlabel(f'{i}')

        plt.suptitle(f'{estuary}: Distribution of {i} Results {filter_title}')
        plt.tight_layout()
        
        output_plot_path = os.path.join(output_folder_path, f'sample_distribution_{i}_{filter_title}_{estuary}.png')
        plt.savefig(output_plot_path, bbox_inches='tight')
        plt.close()

In [16]:
def filter_data(wims_nd_converted, wims_nd_removed, multiple):
    #filtering begins
    filtered_results_nd_removed = []

    chosen_params = wims_nd_removed[wims_nd_removed['param'] != 'Other']['param'].unique() #getting all unique params present in the dataset which
    #are not equal to 'Other'

    for param_value in chosen_params:
        subset = wims_nd_removed[wims_nd_removed['param'] == param_value] #subsetting wims while iterating through the chosen_params
    
        subset_std = subset['result'].std() * multiple
    
        subset_mean = subset['result'].mean()
    
        min_filter = subset_mean - subset_std
        max_filter = subset_mean + subset_std
    
        filtered_subset = subset[(subset['result'] >= min_filter) & (subset['result'] <= max_filter)]
    
        #add the new filtered subset each time one has been cleaned to the master
        filtered_results_nd_removed.append(filtered_subset)

    filtered_df_nd_removed = pd.concat(filtered_results_nd_removed, ignore_index=True)

    #filtering begins
    filtered_results_nd_converted = []

    chosen_params = wims_nd_converted[wims_nd_converted['param'] != 'Other']['param'].unique() #getting all unique params present in the dataset which
    #are not equal to 'Other'

    for param_value in chosen_params:
        subset = wims_nd_converted[wims_nd_converted['param'] == param_value] #subsetting wims while iterating through the chosen_params
    
        subset_std = subset['result'].std() * multiple
    
        subset_mean = subset['result'].mean()
    
        min_filter = subset_mean - subset_std
        max_filter = subset_mean + subset_std
    
        filtered_subset = subset[(subset['result'] >= min_filter) & (subset['result'] <= max_filter)]
    
        #add the new filtered subset each time one has been cleaned to the master
        filtered_results_nd_converted.append(filtered_subset)
    
    filtered_df_nd_converted = pd.concat(filtered_results_nd_converted, ignore_index=True)

    return filtered_df_nd_removed, filtered_df_nd_converted

In [17]:
# Assuming you have a CSV file with multiple columns
csv_file_path = "C:\\Users\\alechutchings\\Documents\\PythonNotebooks\\estuary_shapefile_dictionary.csv"

# Initialize an empty dictionary
data_dict = {}

# Read the CSV file and populate the dictionary
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    
    # Skip the header row if it exists
    next(csv_reader, None)
    
    # Iterate through rows and add key-value pairs to the dictionary
    for row in csv_reader:
        key = row[0]
        values = row[1:]  # Collect all values from the second column onwards
        data_dict[key] = values



In [18]:
estuary_list = list(data_dict.keys())
print(len(estuary_list))

83


In [19]:
wfd_trac = "C:\\Users\\alechutchings\\Documents\\PythonNotebooks\\WFD_trac_shapefile\\WFD_Transitional_and_Coastal_Water_Bodies_Cycle_2.shp"
wfd_trac_shp = gpd.read_file(wfd_trac) 

In [20]:
extra_sample_points = pd.read_csv("C:\\Users\\alechutchings\\Documents\\PythonNotebooks\\Extra_sample_points.csv")
values_to_remove = extra_sample_points['Pointcode'].tolist()
#values_to_remove

In [26]:
longest_line_df = pd.read_csv("C:\\Users\\alechutchings\\Documents\\PythonNotebooks\\Longest_lines_df.csv")
# Convert DataFrame to GeoDataFrame
geometry = [LineString([(x1, y1), (x2, y2)]) for x1, y1, x2, y2 in zip(longest_line_df['CoordA_X'], longest_line_df['CoordA_Y'], longest_line_df['CoordB_X'], longest_line_df['CoordB_Y'])]
longest_line_list_gdf = gpd.GeoDataFrame(longest_line_df, geometry=geometry)
longest_line_list_gdf.crs = 'epsg:27700'
# Convert to latitude and longitude (EPSG 4326)
longest_line_list_gdf_latlon = longest_line_list_gdf.to_crs('epsg:4326')

# Import all WIMS files

In [27]:
import traceback
#reading DETCODE as a string automatically is important as leading zeros get lost if they're read as floats
#added the dtypes argument b/c columns have mixed datatypes and its energy intensive for pandas to guess
#estuary_list = ['Nene','Nene']

#estuary_list = list(data_dict.keys())
estuary_list = ['Dart','Deben']

# Initialize an empty DataFrame to store coordinates
all_longest_line_coords = []

for estuary in estuary_list:
    try:
        #Set up the folder path
        folder_path = f"C:\\Users\\alechutchings\\Documents\\PythonNotebooks\\SeasonalEstuaryChanges\\{estuary}"
        #Create an output plot folder if 
        folder_name = 'output_plots'
        output_path = os.path.join(folder_path, folder_name)
    
        if not os.path.exists(output_path):
            os.mkdir(output_path)
            print(f"Folder '{output_path}' created.")
        else:
            print(f"Folder '{output_path}' already exists.")

        output_folder_path = os.path.join(folder_path,folder_name)
    
        #Import stage
        folder_path_rawdata = os.path.join(folder_path, 'rawdata')
        file_names = [file for file in os.listdir(folder_path_rawdata) if file.endswith('.csv')]
        number_files = len(file_names)
        full_wims_all = import_csv(number_files, folder_path_rawdata, file_names)

        #Remove accidentally added ptcodes
        full_wims_all['PTCODE'] = full_wims_all['PTCODE'].astype(str)
        full_wims_all['PTCODE'] = full_wims_all['PTCODE'].str.strip()
        unique_values_full_wims_all = full_wims_all['PTCODE'].unique()
        print(len(unique_values_full_wims_all))
        full_wims_all_removed_pointcodes = full_wims_all[~full_wims_all['PTCODE'].isin(values_to_remove)]
        unique_values_full_wims_all_removed_pointcodes = full_wims_all_removed_pointcodes['PTCODE'].unique()
        print(len(unique_values_full_wims_all_removed_pointcodes))

        #duplicate row check
        full_wims_no_dup = remove_duplicate_rows(full_wims_all_removed_pointcodes)
        
        full_wims_no_dup.columns = full_wims_no_dup.columns.str.lower() #converts column names to lower case

        #List of accepting purpose codes
        accepted_pcs = ['MS',
                        'MP',
                        'MN',
                        'MU']
        # Filter the DataFrame based on 'purpose' column
        full_wims_no_dup_accepted_purp = full_wims_no_dup[full_wims_no_dup['purpose'].isin(accepted_pcs)]

        #folder_path_shapefile = os.path.join(folder_path, 'shapefile')
        shapefile = extract_shapefile(data_dict, estuary, wfd_trac_shp, target_crs='EPSG:27700')
        #Lat long crs
        target_crs = "EPSG:4326"
    
        # Reproject the GeoDataFrame to the target CRS
        est_shape = shapefile.to_crs(target_crs)
        
        if estuary in longest_line_list_gdf_latlon['Estuary'].values:
            longest_line = longest_line_list_gdf_latlon[longest_line_list_gdf_latlon['Estuary'] == estuary]
            print(f"{estuary} longest line extracted from table")
        else:
            longest_line = find_longest_line(est_shape)
            print(f"{estuary} longest line extracted from shapefile")

        # Extract coordinates of the longest line
        longest_line_coords = []
        for point in longest_line['geometry'].iloc[0].coords:
            lon, lat = point
            longest_line_coords.append({'Estuary': estuary, 'Longitude': lon, 'Latitude': lat})
        
        # Convert the coordinates list to a DataFrame and append to the list
        longest_line_coords_df = pd.DataFrame(longest_line_coords)
        all_longest_line_coords.append(longest_line_coords_df)

            
        full_wims_reproj = redefine_site_number(estuary, full_wims_no_dup_accepted_purp, longest_line, est_shape, output_folder_path)
        full_wims_dt = convert_date_times(full_wims_reproj)
        full_wims_surface_samples = screen_sample_depths(estuary,full_wims_dt,output_folder_path)
        full_wims_seasons = get_seasons(full_wims_surface_samples)
        wims_sal_filled_renamed = assign_average_salinity(estuary,full_wims_seasons,output_folder_path)
        wims_nd_converted, wims_nd_removed = generate_seperate_dataframes(wims_sal_filled_renamed)
        plot_all_variables(estuary, wims_nd_converted, wims_nd_removed, colours, output_folder_path, 'before filtering')
        filtered_df_nd_removed, filtered_df_nd_converted = filter_data(wims_nd_converted, wims_nd_removed,4)
        plot_all_variables(estuary, filtered_df_nd_removed, filtered_df_nd_converted, colours, output_folder_path, 'after filtering')
    
        output_csv_nd_converted_path = os.path.join(folder_path, f'{estuary}_wims_data_clean_nd_converted.csv')
        filtered_df_nd_converted.to_csv(output_csv_nd_converted_path)
        output_csv_nd_removed_path = os.path.join(folder_path, f'{estuary}_wims_data_clean_nd_removed.csv')
        filtered_df_nd_removed.to_csv(output_csv_nd_removed_path)
        print(f'{estuary}_process_completed')
    
    except Exception as e:
        # Print the error message
        error_message = f"Error processing {estuary}: {str(e)}"

        # Log the error to a notepad file
        log_file_path = os.path.join('C:\\Users\\alechutchings\\Documents\\PythonNotebooks\\SeasonalEstuaryChanges\\', 'error_log_cleaning_04_03_2024.txt')
        with open(log_file_path, 'a') as log_file:
            log_file.write(error_message + '\n')
        print(error_message)

    finally:
        # Any cleanup code or additional actions you want to perform regardless of success or failure
        pass



Folder 'C:\Users\alechutchings\Documents\PythonNotebooks\SeasonalEstuaryChanges\Dart\output_plots' already exists.
80
79
No duplicate rows found.
            wb_id wb_name  rbd_id    rbd_name        wb_cat    st_area_sh  \
0  GB510804605900    DART     8.0  South West  Transitional  8.319579e+06   

     st_perimet                                           geometry  
0  63615.157234  POLYGON ((284859.519 56881.427, 284839.000 568...  
Dart longest line extracted from shapefile


C:\Users\alechutchings\AppData\Local\Temp\2\ipykernel_6344\3976447997.py:33: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lambda x: longest_line["geometry"].distance(x).min()


147318
Dates converted successfully!
123339
89511


C:\Users\alechutchings\AppData\Local\Temp\2\ipykernel_6344\1041832945.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wims_nd_removed['param'] = 'Other'
C:\Users\alechutchings\AppData\Local\Temp\2\ipykernel_6344\1041832945.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wims_nd_removed.rename(columns={ 'site_number_x':'site_number'},inplace=True)


Dart_process_completed
Folder 'C:\Users\alechutchings\Documents\PythonNotebooks\SeasonalEstuaryChanges\Deben\output_plots' already exists.


...

23
23
No duplicate rows found.
            wb_id wb_name  rbd_id rbd_name        wb_cat    st_area_sh  \
0  GB520503503900   DEBEN     5.0  Anglian  Transitional  7.819426e+06   

     st_perimet                                           geometry  
0  68304.521348  POLYGON ((629472.187 250426.187, 629499.026 25...  
Deben longest line extracted from table


C:\Users\alechutchings\AppData\Local\Temp\2\ipykernel_6344\3976447997.py:33: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lambda x: longest_line["geometry"].distance(x).min()


13895
Dates converted successfully!
13839
12713


C:\Users\alechutchings\AppData\Local\Temp\2\ipykernel_6344\1041832945.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wims_nd_removed['param'] = 'Other'
C:\Users\alechutchings\AppData\Local\Temp\2\ipykernel_6344\1041832945.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wims_nd_removed.rename(columns={ 'site_number_x':'site_number'},inplace=True)


Deben_process_completed


In [28]:
all_longest_line_coords_df = pd.concat(all_longest_line_coords, ignore_index=True)
all_longest_line_coords_df.to_csv("all_longest_line_coords_dart_deben.csv", index=False)